In [1]:
import warnings
warnings.filterwarnings('ignore')

from gensim.models import Word2Vec
import gensim
import pandas as pd


In [2]:
import os
import re
import io
import sys
import math
import json
import html
import pickle
import difflib
import unicodedata
import bs4
import bz2
import py7zr
import numpy
import mwparserfromhell
import libarchive.public
#import dataset
import wikitextparser as wtp
from pprint import pprint
from wikitextparser import remove_markup, parse
import datetime
from pandas import DataFrame
import random
import pickle
import pandas as pd
import logging
from datasets import *
from models import *

In [3]:
flight_fds_clean=[['flight','act_dep_time'],['flight','sched_dep_time'],['flight','sched_arr_time'],['flight','act_arr_time']]

In [64]:
flight_fds_dirty=[['flight','act_dep_time'],['flight','sched_dep_time'],['flight','sched_arr_time'],['flight','act_arr_time']]

In [5]:
flights=("datasets/flights/dirty.csv")
flights_clean=("datasets/flights/clean.csv")
flights_df=read_csv_dataset(flights)
flights_clean_df=read_csv_dataset(flights_clean)

In [8]:
#flights_clean_df.values.tolist()


In [6]:
def read_csv_dataset(dataset_path):
        """
        This method reads a dataset from a csv file path.
        """
        dataframe = pd.read_csv(dataset_path,index_col=0)
        return dataframe

In [7]:
def prepare_dataset_for_retrain_realworld(clean_data_df, dirty_data_df): #send two datasets
        clean_data = clean_data_df.applymap(str)
        dirty_data = dirty_data_df.applymap(str)
        if clean_data.shape != dirty_data.shape:
            sys.stderr.write("Two compared datasets do not have equal sizes!\n")
        else:
            clean_data_col=clean_data.columns.values
            dirty_data_col=dirty_data.columns.values
            for dir_col, clean_col in zip(dirty_data_col, clean_data_col):
                dirty_data.loc[dirty_data[dir_col] != clean_data[clean_col], dir_col] = None #replace error value with NaN
        return dirty_data

In [68]:
prepare_dirty=prepare_dataset_for_retrain_realworld(flights_clean_df,flights_df)
flight_fds_dirty=[['flight','act_dep_time','sched_dep_time'],['flight','sched_arr_time','act_arr_time']]

In [71]:
#flights_df
want_to_repair=0
correct_c=0
for fds in flight_fds_dirty:
    prepare_dirty_df=prepare_dirty[fds]
    flights_fds=flights_df[fds]
    flight_fds_clean=flights_clean_df[fds]
    data_for_retrain=prepare_dirty_df.values.tolist()
    prepare_dirty_df=prepare_dirty_df.values.tolist()
    flight_fds_clean=flight_fds_clean.values.tolist()

    train_data_rows=[]
    for row in data_for_retrain:
        row = list(map(str, row))
        row=list(filter(None, row))
        if len(row)<3:
            continue
        else:
            train_data_rows.append(row)
    #print(train_data_rows)
    #model=FastText(train_data_rows, min_count=1, workers=8, iter=1000, window=2, sg=1, word_ngrams=1)
    model = Word2Vec(train_data_rows, sg=1, min_count=1, workers=8, iter=1000, window=3)
    word_vectors = model.wv
    for row, row_a in zip(prepare_dirty_df,flight_fds_clean):
        #print(row)
        row1=str(row)
        if "None" in row1:
            want_to_repair=want_to_repair+1
            row=list(filter(None, row))
            errorneous_value=row[0]
            #print(errorneous_value)
            errorneous_value=str(errorneous_value)
            #print(errorneous_value)
            #if errorneous_value in word_vectors.vocab:
            try:
                #print("yes")
                if len(row)==2:
                    want_to_repair=want_to_repair+1
                    #R_Value=model.most_similar(errorneous_value)
                    print(row)
                    R_Value=model.wv.most_similar(positive=row)
                    #print(R_Value)
                    a=R_Value[0]
                    b,c=a
                   # print(b,row_a[1])
                    if len(b)>11:
                         want_to_repair=want_to_repair-1
                    else:

                        if b==row_a[1]:
                            correct_c=correct_c+1
            except:
                print("Word not in the vocabulary")
print(correct_c,want_to_repair)
print(correct_c/want_to_repair)
            
            
        #have_to_clean
        
    
            #print(train_data_rows)
#print(train_data_rows)

['AA-1640-MIA-MCO', '6:30 p.m.']
['AA-204-LAX-MCO', '11:25 p.m.']
['AA-1522-SFO-ORD', '11:55 p.m.']
['AA-3823-LAX-DEN', '9:00 p.m.']
['AA-2312-DFW-DTW', '8:25 p.m.']
['AA-649-ORD-SNA', '1:30 p.m.']
['AA-3859-IAH-ORD', '7:16 a.m.']
['CO-1586-IAH-MCO', '7:00 p.m.']
['AA-518-MIA-JFK', '6:54 a.m.']
['CO-58-DEN-IAH', '11:25 a.m.']
['CO-1614-LAX-IAH', '12:30 a.m.']
['CO-52-IAH-LAX', '7:35 a.m.']
['CO-1090-BOS-IAH', '9:45 a.m.']
['AA-3468-CVG-MIA', '7:25 a.m.']
['AA-484-DFW-MIA', '4:29 p.m.']
['CO-62-IAH-EWR', '2:30 p.m.']
['CO-1561-PHX-IAH', '8:05 a.m.']
['CO-1023-IAH-DEN', '11:40 a.m.']
['CO-89-IAH-EWR', '6:00 a.m.']
['CO-1088-CLE-IAH', '8:40 a.m.']
['CO-63-EWR-IAH', '5:25 p.m.']
['CO-47-IAH-LAX', '7:10 p.m.']
['AA-4330-CVG-ORD', '3:35 p.m.']
['CO-1694-LAX-IAH', '7:15 p.m.']
['CO-1193-EWR-MCO', '9:15 a.m.']
['AA-2525-DFW-MIA', '7:39 a.m.']
['AA-1221-MCO-ORD', '8:23 p.m.']
['CO-16-LAX-EWR', '10:25 a.m.']
['UA-37-EWR-MCO', '1:29 p.m.']
['CO-1250-MIA-IAH', '2:53 p.m.']
['UA-828-SFO-ORD', '11:0

['AA-484-DFW-MIA', '4:15 p.m.']
['AA-1886-BOS-MIA', '10:55 a.m.']
['AA-446-DFW-PHL', '11:50 a.m.']
['AA-466-IAH-MIA', '6:00 a.m.']
['UA-3050-PHX-CLT', '11:05 a.m.']
['AA-1664-MIA-ATL', '10:15 a.m.']
['UA-257-JFK-SFO', '2:30 p.m.']
['AA-2050-ORD-MIA', '10:40 a.m.']
['AA-4330-CVG-ORD', '3:36 p.m.']
['AA-1434-DFW-MCO', '7:15 a.m.']
['AA-1640-MIA-MCO', '6:30 p.m.']
['AA-4307-ORD-DTW', '6:45 p.m.']
['CO-1694-LAX-IAH', '7:15 p.m.']
['CO-1193-EWR-MCO', '9:15 a.m.']
['AA-3842-MSP-ORD', '6:00 a.m.']
['AA-643-MIA-ORD', '7:10 a.m.']
['UA-2906-PHL-MCO', '3:50 p.m.']
['AA-400-DFW-JFK', '10:40 a.m.']
['UA-664-ORD-PHL', '1:33 p.m.']
['AA-3756-ORD-SLC', '12:15 p.m.']
['AA-1522-SFO-ORD', '11:55 p.m.']
['UA-233-LAX-JFK', '4:25 p.m.']
['AA-3823-LAX-DEN', '9:00 p.m.']
['AA-2312-DFW-DTW', '8:25 p.m.']
['AA-1165-JFK-MIA', '6:59 a.m.']
['AA-431-MIA-SFO', '8:35 a.m.']
['AA-649-ORD-SNA', '1:30 p.m.']
['UA-828-SFO-ORD', '11:08 p.m.']
['CO-4888-IAH-DAL', '5:30 p.m.']
['AA-1917-JFK-MCO', '2:55 p.m.']
['AA-2268-PH

['AA-643-MIA-ORD', '7:10 a.m.']
['UA-2906-PHL-MCO', '3:50 p.m.']
['UA-664-ORD-PHL', '1:33 p.m.']
['UA-37-EWR-MCO', '1:29 p.m.']
['UA-233-LAX-JFK', '4:25 p.m.']
['AA-3823-LAX-DEN', '9:00 p.m.']
['AA-2312-DFW-DTW', '8:25 p.m.']
['AA-649-ORD-SNA', '1:30 p.m.']
['UA-828-SFO-ORD', '11:08 p.m.']
['CO-4888-IAH-DAL', '5:30 p.m.']
['AA-3063-SLC-LAX', '8:20 p.m.']
['AA-2268-PHX-ORD', '7:15 a.m.']
['AA-222-LAX-BOS', '7:10 a.m.']
['AA-4198-ORD-CLE', '10:40 a.m.']
['CO-45-EWR-MIA', '4:00 p.m.']
['AA-1640-MIA-MCO', '6:30 p.m.']
['CO-1694-LAX-IAH', '7:15 p.m.']
['UA-2830-MCO-CLT', '3:27 p.m.']
['AA-204-LAX-MCO', '11:25 p.m.']
['UA-2726-FLL-PHL', '4:00 p.m.']
['AA-1522-SFO-ORD', '11:55 p.m.']
['UA-233-LAX-JFK', '4:25 p.m.']
['AA-3823-LAX-DEN', '9:00 p.m.']
['AA-2312-DFW-DTW', '8:25 p.m.']
['AA-649-ORD-SNA', '1:30 p.m.']
['UA-3099-PHX-PHL', '11:55 a.m.']
['AA-1733-ORD-PHX', '10:30 p.m.']
['AA-1640-MIA-MCO', '7:25 p.m.']
['AA-2312-DFW-DTW', '11:50 p.m.']
['AA-649-ORD-SNA', '3:50 p.m.']
['AA-3063-SLC-LAX

['CO-58-DEN-IAH', '3:31 p.m.']
['UA-5487-SFO-MRY', '11:21 a.m.']
['UA-414-LAX-SFO', '7:17 a.m.']
['CO-1614-LAX-IAH', '5:26 a.m.']
['UA-2726-FLL-PHL', '6:46 p.m.']
['CO-1090-BOS-IAH', '1:19 p.m.']
['AA-616-DFW-DTW', '12:35 p.m.']
['UA-3515-IAD-MSP', '9:56 a.m.']
['AA-1886-BOS-MIA', '1:40 p.m.']
['CO-62-IAH-EWR', '7:10 p.m.']
['CO-1561-PHX-IAH', '11:32 a.m.']
['CO-1023-IAH-DEN', '1:14 p.m.']
['UA-3050-PHX-CLT', '4:57 p.m.']
['CO-89-IAH-EWR', '10:27 a.m.']
['CO-1088-CLE-IAH', '10:56 a.m.']
['AA-1664-MIA-ATL', '12:10 p.m.']
['AA-1279-DFW-PHX', '2:35 p.m.']
['UA-854-SFO-IAH', '1:30 p.m.']
['UA-382-IAD-LAX', '3:21 p.m.']
['UA-2945-PHL-CLT', '1:45 p.m.']
['CO-63-EWR-IAH', '8:03 p.m.']
['UA-248-PHX-ORD', '5:34 p.m.']
['AA-1434-DFW-MCO', '10:35 a.m.']
['AA-1640-MIA-MCO', '7:25 p.m.']
['AA-4307-ORD-DTW', '9:10 p.m.']
['UA-843-LAX-ORD', '7:58 p.m.']
['AA-643-MIA-ORD', '9:35 a.m.']
['UA-2906-PHL-MCO', '6:23 p.m.']
['UA-938-DEN-ORD', '7:36 p.m.']
['AA-1221-MCO-ORD', '9:45 p.m.']
['CO-16-LAX-EWR', '

['CO-16-LAX-EWR', '6:34 p.m.']
['AA-3756-ORD-SLC', '2:45 p.m.']
['UA-37-EWR-MCO', '4:17 p.m.']
['AA-2312-DFW-DTW', '11:50 p.m.']
['UA-3373-IAD-PHL', '9:13 a.m.']
['AA-649-ORD-SNA', '3:50 p.m.']
['CO-1250-MIA-IAH', '4:59 p.m.']
['UA-6273-YYC-SFO', '8:45 a.m.']
['CO-4888-IAH-DAL', '6:40 p.m.']
['UA-858-PVG-SFO', '8:02 a.m.']
['UA-62-IAH-EWR', '7:10 p.m.']
['AA-3804-PHL-ORD', '4:05 p.m.']
['AA-1917-JFK-MCO', '5:55 p.m.']
['AA-2268-PHX-ORD', '11:35 a.m.']
['AA-2957-DFW-CVG', '11:05 a.m.']
['UA-37-EDI-EWR', '12:09 p.m.']
['AA-789-ORD-DEN', '2:35 p.m.']
['AA-3979-CVG-ORD', '8:00 a.m.']
['AA-1544-SAN-ORD', '4:56 p.m.']
['AA-3786-IAH-ORD', '6:40 p.m.']
['CO-50-CLE-EWR', '5:26 p.m.']
['UA-3099-PHX-PHL', '6:17 p.m.']
['AA-4198-ORD-CLE', '12:55 p.m.']
['AA-404-MIA-MCO', '7:45 a.m.']
['AA-204-LAX-MCO', '6:55 a.m.']
['AA-1640-MIA-MCO', '7:25 p.m.']
['AA-1733-ORD-PHX', '10:30 p.m.']
['AA-1640-MIA-MCO', '7:25 p.m.']
['CO-1694-LAX-IAH', '12:21 a.m.']
['AA-204-LAX-MCO', '6:55 a.m.']
['AA-1279-DFW-PHX',

In [62]:
print(correct_c,want_to_repair)

877 7311


In [21]:
train_data_rows=[]
prepare_dirty=prepare_dataset_for_retrain_realworld(flights_clean_df,flights_df)
for fds in flight_fds_clean:
    prepare_dirty=prepare_dirty[fds]
    flights=flights[fds]
    flight_fds_dirty=flight_fds_dirty[fds]

data_for_retrain=prepare_dirty.values.tolist()
for row in data_for_retrain:
    row = list(map(str, row))
    row=list(filter(None, row))
    if not any(x1 in "None" for x1 in row):
        train_data_rows.append(row)
if train_data_rows:
    print(train_data_rows)

TypeError: string indices must be integers

In [113]:
#prepare_dirty

In [78]:
model = Word2Vec(train_data_rows, sg=1, min_count=1, workers=8, iter=1000, window=len(train_data_rows[0]))

In [81]:
#model.most_similar("aa AA-1733-ORD-PHX")

In [97]:
import warnings
warnings.filterwarnings('ignore')
import fasttext
from fasttext import train_unsupervised
import gensim
from gensim.models import FastText

In [98]:
model_hosp1 = FastText(train_data_rows, min_count=1, workers=8, iter=1000, window=1, sg=1, word_ngrams=1)

In [103]:
model_hosp1.most_similar("AA-3859-IAH-ORD 9:40 a.m. 9:32 a.m.")

[('aa AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.762505292892456),
 ('quicktrip AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.7548702359199524),
 ('flylouisville AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.7273005247116089),
 ('flights AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.7242389917373657),
 ('flightstats AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.7042007446289062),
 ('businesstravellogue AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.7038495540618896),
 ('world-flight-tracker AA-3859-IAH-ORD 7:10 a.m. 7:16 a.m. 9:40 a.m. 9:32 a.m.',
  0.6343488693237305),
 ('quicktrip AA-466-IAH-MIA 6:00 a.m. 6:08 a.m. 9:20 a.m. 9:05 a.m.',
  0.6072946786880493),
 ('aa AA-466-IAH-MIA 6:00 a.m. 6:08 a.m. 9:20 a.m. 9:05 a.m.',
  0.5790500640869141),
 ('ua UA-3373-IAD-PHL 8:29 a.m. 8:26 a.m. 9:24 a.m. 9:13 a.m.',
  0.5732736587524414)]

In [164]:
prepare_dirty_k=prepare_dataset_for_retrain_realworld(flights_clean_df,flights_df)
prepare_dirty_k=prepare_dirty_k.values.tolist()
flights_clean_df_l=flights_clean_df.values.tolist()
want_to_repair=0
correct_c=0
for row, row_c in zip(prepare_dirty_k,flights_clean_df_l):
    row = list(map(str, row))
    row=list(filter(None, row))
    #print(row)
    clean_row=[]
    if "None" in row:
        dirty=row
        row = [e for e in row if e not in "None"]
        row=row[1:]
        value=[' '.join(row)]
        #print(value)
        similar=model_hosp1.most_similar(value)
        a=similar[0]
        b,c=a
        correct=b.split()
        clean_row.append(correct[1])
        join=correct[2]+' '+correct[3]
        clean_row.append(join)
        join=correct[4]+' '+correct[5]
        clean_row.append(join)
        join=correct[6]+' '+correct[7]
        clean_row.append(join)
        join=correct[8]+' '+correct[9]
        clean_row.append(join)
        #print(dirty,clean_row,row_c)
        dirty=dirty[1:]
        flag=False
        row_c=row_c[2:]
        #print(row_c)
        for value, value_c, row_a in  zip(dirty,clean_row, row_c):
            #print(value)
           
            if dirty[0]==clean_row[0]:
                print(dirty[0],clean_row[0] )
                #print("yes")
                flag=True
            value=str(value)
            if value=="None" and flag:
                #print("yes")
                want_to_repair=want_to_repair+1
                
                if row_a==value_c:
                    print(row_a, value_c)
                    correct_c=correct_c+1
print(correct_c,want_to_repair)                    
print(correct_c/want_to_repair)
            
       # break
        
    

AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
AA-518-MIA-JFK AA-518-MIA-JFK
AA-518-MIA-JFK AA-518-MIA-JFK
AA-518-MIA-JFK AA-518-MIA-JFK
AA-518-MIA-JFK AA-518-MIA-JFK
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-2830-MCO-CLT UA-2830-MCO-CLT
UA-2830-MCO-CLT UA-2830-MCO-CLT
3:27 p.m. 3:27 p.m.
UA-2830-MCO-CLT UA-2830-MCO-CLT
UA-2830-MCO-CLT UA-2830-MCO-CLT
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
UA-5487-SFO-MRY UA-5487-SFO-MRY
UA-5487-SFO-MRY UA-5487-SFO-MRY
UA-5487-SFO-MRY UA-5487-SFO-MRY
UA-5487-SFO-MRY UA-5

AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
11:55 a.m. 11:55 a.m.
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA

AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
AA-3809-PHX-LAX AA-3809-

UA-62-IAH-EWR UA-62-IAH-EWR
UA-62-IAH-EWR UA-62-IAH-EWR
UA-62-IAH-EWR UA-62-IAH-EWR
UA-62-IAH-EWR UA-62-IAH-EWR
AA-3804-PHL-ORD AA-3804-PHL-ORD
AA-3804-PHL-ORD AA-3804-PHL-ORD
AA-3804-PHL-ORD AA-3804-PHL-ORD
AA-3804-PHL-ORD AA-3804-PHL-ORD
AA-1917-JFK-MCO AA-1917-JFK-MCO
AA-1917-JFK-MCO AA-1917-JFK-MCO
AA-1917-JFK-MCO AA-1917-JFK-MCO
AA-1917-JFK-MCO AA-1917-JFK-MCO
AA-2268-PHX-ORD AA-2268-PHX-ORD
AA-2268-PHX-ORD AA-2268-PHX-ORD
AA-2268-PHX-ORD AA-2268-PHX-ORD
AA-2268-PHX-ORD AA-2268-PHX-ORD
UA-2314-ATL-PHL UA-2314-ATL-PHL
UA-2314-ATL-PHL UA-2314-ATL-PHL
UA-2314-ATL-PHL UA-2314-ATL-PHL
UA-2314-ATL-PHL UA-2314-ATL-PHL
AA-4277-CVG-JFK AA-4277-CVG-JFK
AA-4277-CVG-JFK AA-4277-CVG-JFK
AA-4277-CVG-JFK AA-4277-CVG-JFK
AA-4277-CVG-JFK AA-4277-CVG-JFK
AA-2957-DFW-CVG AA-2957-DFW-CVG
AA-2957-DFW-CVG AA-2957-DFW-CVG
AA-2957-DFW-CVG AA-2957-DFW-CVG
AA-2957-DFW-CVG AA-2957-DFW-CVG
UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
AA-789-O

AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
AA-3859-IAH-ORD AA-3859-IAH-ORD
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
CO-1586-IAH-MCO CO-1586-IAH-MCO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-397-JFK-SFO UA-397-JFK-SFO
UA-3925-IAD-ORF UA-3925-IAD-ORF


...

AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-271-MIA-LAX AA-271-MIA-LAX
AA-3842-MSP-ORD AA-3842-MSP-ORD
AA-3842-MSP-ORD AA-3842-MSP-ORD
AA-3842-MSP-ORD AA-3842-MSP-ORD
AA-3842-MSP-ORD AA-3842-MSP-ORD
AA-1007-MIA-PHX AA-1007-MIA-PHX
AA-1007-MIA-PHX AA-1007-MIA-PHX
AA-1007-MIA-PHX AA-1007-MIA-PHX
AA-1007-MIA-PHX AA-1007-MIA-PHX
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-2525-DFW-MIA AA-2525-DFW-MIA
AA-1221-MCO-ORD AA-1221-MCO-ORD
AA-1221-MCO-ORD AA-1221-MCO-ORD
AA-1221-MCO-ORD AA-1221-MCO-ORD
AA-1221-MCO-ORD AA-1221-MCO-ORD
AA-4307-ORD-DTW AA-4307-ORD-DTW
AA-4307-ORD-DTW AA-4307-ORD-DTW
AA-4307-ORD-DTW AA-4307-ORD-DTW
AA-4307-ORD-DTW AA-4307-ORD-DTW
AA-1165-JFK-MIA AA-1165-JFK-MIA
AA-1165-JFK-MIA AA-1165-JFK-MIA
AA-1165-JFK-MIA AA-1165-JFK-MIA
AA-1165-JFK-MIA AA-1165-JFK-MIA
AA-431-MIA-SFO AA-431-MIA-SFO
AA-431-MIA-SFO AA-431-MIA-SFO
AA-431-MIA-SFO AA-431-MIA-SFO
AA-431-MIA-SFO AA-431-MIA-SFO
UA-257-JFK-SFO UA-257-

AA-789-ORD-DEN AA-789-ORD-DEN
AA-85-JFK-SFO AA-85-JFK-SFO
AA-85-JFK-SFO AA-85-JFK-SFO
AA-85-JFK-SFO AA-85-JFK-SFO
AA-85-JFK-SFO AA-85-JFK-SFO
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-3786-IAH-ORD AA-3786-IAH-ORD
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
AA-222-LAX-BOS AA-222-LAX-BOS
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
AA-4198-ORD-CLE AA-4198-ORD-CLE
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
CO-45-EWR-MIA CO-45-EWR-MIA
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-3809-PHX-LAX AA-3809-PHX-LAX
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
AA-59-JFK-SFO AA-59-JFK-SFO
UA

CO-1088-CLE-IAH CO-1088-CLE-IAH
CO-1088-CLE-IAH CO-1088-CLE-IAH
8:40 a.m. 8:40 a.m.
CO-1088-CLE-IAH CO-1088-CLE-IAH
CO-1088-CLE-IAH CO-1088-CLE-IAH
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
UA-854-SFO-IAH UA-854-SFO-IAH
UA-854-SFO-IAH UA-854-SFO-IAH
UA-854-SFO-IAH UA-854-SFO-IAH
UA-854-SFO-IAH UA-854-SFO-IAH
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
AA-4344-ORD-DTW AA-4344-ORD-DTW
AA-4344-ORD-DTW AA-4344-ORD-DTW
AA-4344-ORD-DTW AA-4344-ORD-DTW
AA-4344-ORD-DTW AA-4344-ORD-DTW
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-2945-PHL-CLT UA-2945-PHL-CLT
UA-2945-PHL-CLT UA-2945-PHL-CLT
UA-2945-PHL-CLT UA-2945-PHL-CLT
UA-2945-PHL-

UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
UA-37-EDI-EWR UA-37-EDI-EWR
AA-415-BOS-ORD AA-415-BOS-ORD
AA-415-BOS-ORD AA-415-BOS-ORD
AA-415-BOS-ORD AA-415-BOS-ORD
AA-415-BOS-ORD AA-415-BOS-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
AA-1544-SAN-ORD AA-1544-SAN-ORD
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
CO-50-CLE-EWR CO-50-CLE-EWR
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
UA-3099-PHX-PHL UA-3099-PHX-PHL
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
CO-58-DEN-IAH CO-58-DEN-IAH
CO-1023-IAH-DEN CO-1023-IAH-DEN
CO-1023-IAH-DEN CO-1023-IAH-DEN
CO-1023-IAH-DEN CO-1023-IAH-DEN
CO-1023-IAH-DEN CO-1023-IAH-DEN
UA-938-DEN-ORD UA-938-DEN-ORD
UA-938-DEN-ORD UA-938-DEN-ORD
UA-938-DEN-ORD UA-938-DEN-ORD
UA-938-DEN-ORD UA-938-DEN-ORD
AA-789-ORD-DEN AA-789-ORD-DEN
AA-789-ORD

AA-2957-DFW-CVG AA-2957-DFW-CVG
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-1664-MIA-ATL AA-1664-MIA-ATL
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-3979-CVG-ORD AA-3979-CVG-ORD
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-1279-DFW-PHX AA-1279-DFW-PHX
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-404-MIA-MCO AA-404-MIA-MCO
AA-3756-ORD-SLC AA-3756-ORD-SLC
AA-3756-ORD-SLC AA-3756-ORD-SLC
AA-3756-ORD-SLC AA-3756-ORD-SLC
AA-3756-ORD-SLC AA-3756-ORD-SLC
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
UA-382-IAD-LAX UA-382-IAD-LAX
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-257-JFK-SFO UA-257-JFK-SFO
UA-2945-PHL-CLT UA-2945-PHL-CLT
UA-2945-PHL-CLT UA-2945-PHL-CLT
UA-2945-PHL-CLT UA-2945-PHL-CLT


In [ ]:
if "None" in row:
        clean=row
        row = [e for e in row if e not in "None"]
        print(row)
        similar=model_hosp1.most_similar(row)
        a=similar[0]
        b,c=a
        correct=b.split()
        print(correct)
        
        